In [1]:
import pandas as pd
import numpy as np

In [2]:
movies = pd.read_csv("../sample_data/movies.csv")
ratings = pd.read_csv("../sample_data/ratings.csv")

In [3]:
movies["genres"] = movies["genres"].apply(lambda x: "etc" if x == "(no genres listed)" else x)
genres_columns = movies["genres"].str.get_dummies('|').columns
genres_columns

Index(['Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime',
       'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'IMAX',
       'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western',
       'etc'],
      dtype='object')

In [4]:
movies = pd.concat([movies.loc[:, ["movieId", "title"]], movies["genres"].str.get_dummies('|')], axis=1)
movies.head()

,movieId,title,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,...,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,etc
0,1,Toy Story (1995),0,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji (1995),0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men (1995),0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,4,Waiting to Exhale (1995),0,0,0,0,1,0,0,1,...,0,0,0,0,1,0,0,0,0,0
4,5,Father of the Bride Part II (1995),0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
np_ratings = ratings.groupby("userId").count()["rating"].to_numpy()
np_ratings = np.sort(np_ratings)

lower_bound = np_ratings[int((len(np_ratings)*0.2))]
upper_bound = np_ratings[int((len(np_ratings)*0.8))]

In [6]:
temp_group = ratings.groupby("userId").count()
filted_userId = temp_group.loc[(temp_group["rating"] > lower_bound) & (temp_group["rating"] < upper_bound)].reset_index()
filted_userId = filted_userId[["userId"]]

ratings = pd.merge(left=filted_userId, right=ratings, on="userId")
ratings["userId"].nunique()

148513

In [7]:
userId = list(set(ratings.userId.to_numpy()))


In [8]:
target_userID = np.random.choice(userId, 1)[0]
src_userID = np.random.choice(userId, 1000)

src_userID = np.sort(src_userID)

In [9]:
empty_genres = np.zeros((len(src_userID), len(genres_columns)))

user_pivot_table = pd.DataFrame({"userId":src_userID}).reset_index(drop=True)
user_pivot_table = pd.concat([user_pivot_table, pd.DataFrame(empty_genres, columns=genres_columns).reset_index(drop=True)], axis=1)
user_pivot_table.sort_values(by="userId", inplace=True)
user_pivot_table

,userId,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,...,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,etc
0,201,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,503,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,504,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,804,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,978,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,246964,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
996,246985,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
997,247114,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
998,247265,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
genres = movies.drop(["title", "movieId"], axis=1)
temp = pd.merge(left=ratings, right=movies, on="movieId")

In [11]:

target_user_ratings = temp.loc[temp["userId"] == target_userID][["rating"]].to_numpy()
target_user_genres = temp.loc[temp["userId"] == target_userID][genres_columns].to_numpy()

x = np.sum(target_user_ratings * target_user_genres, axis=0)
target_user_row = (x - np.min(x))/(np.max(x) - np.min(x))

In [12]:
target_user_row = target_user_row.reshape(1, -1)

In [13]:

for id in src_userID:        
    #id, movieId, rate = ratings.loc[idx]["userId"],  ratings.loc[idx]["movieId"], ratings.loc[idx]["rating"]    
    
    rate = temp.loc[temp["userId"] == id][["rating"]].to_numpy()
    genres = temp.loc[temp["userId"] == id][genres_columns].to_numpy()    
    
    x = np.sum(rate * genres, axis=0)
    src_user_row = (x - np.min(x))/(np.max(x) - np.min(x)).reshape(1, -1)
    
    target_user_row = np.concatenate((target_user_row, src_user_row))
    target_user_row

In [14]:
target_user_row.shape

(1001, 20)

In [15]:
target_user_row[1:].shape

(1000, 20)

In [16]:
user_pivot_table

,userId,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,...,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,etc
0,201,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,503,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,504,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,804,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,978,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,246964,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
996,246985,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
997,247114,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
998,247265,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
user_pivot_table.loc[:, genres_columns] = target_user_row[1:]
user_pivot_table

,userId,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,...,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,etc
0,201,0.912752,0.516779,0.026846,0.020134,1.000000,0.375839,0.000000,0.738255,0.194631,...,0.362416,0.000000,0.033557,0.073826,0.328859,0.496644,0.691275,0.073826,0.020134,0.0
1,503,0.356223,0.223176,0.057940,0.096567,0.429185,0.480687,0.000000,1.000000,0.111588,...,0.167382,0.034335,0.038627,0.291845,0.401288,0.141631,0.864807,0.070815,0.000000,0.0
2,504,0.264550,0.116402,0.010582,0.037037,0.423280,0.317460,0.031746,1.000000,0.031746,...,0.132275,0.000000,0.047619,0.275132,0.402116,0.190476,0.582011,0.058201,0.000000,0.0
3,804,0.639344,0.459016,0.180328,0.245902,0.540984,0.540984,0.000000,0.901639,0.114754,...,0.098361,0.180328,0.180328,0.114754,0.377049,0.295082,1.000000,0.180328,0.049180,0.0
4,978,1.000000,0.838710,0.000000,0.000000,0.580645,0.387097,0.000000,0.741935,0.000000,...,0.000000,0.096774,0.000000,0.000000,0.709677,0.225806,0.741935,0.258065,0.225806,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,246964,0.444444,0.777778,0.444444,0.511111,0.911111,0.800000,0.000000,0.866667,0.288889,...,0.222222,0.288889,0.333333,0.222222,0.200000,0.222222,1.000000,0.000000,0.066667,0.0
996,246985,1.000000,0.844444,0.088889,0.066667,0.822222,0.577778,0.000000,0.511111,0.244444,...,0.000000,0.000000,0.088889,0.177778,0.555556,0.666667,0.444444,0.044444,0.355556,0.0
997,247114,0.229730,0.189189,0.432432,0.337838,1.000000,0.283784,0.000000,0.540541,0.067568,...,0.108108,0.000000,0.189189,0.108108,0.189189,0.000000,0.135135,0.000000,0.081081,0.0
998,247265,0.635417,0.447917,0.041667,0.083333,0.375000,0.177083,0.000000,1.000000,0.125000,...,0.093750,0.000000,0.041667,0.031250,0.406250,0.239583,0.145833,0.437500,0.031250,0.0


In [18]:
user_pivot_table["userId"].iloc[0]

201

In [19]:
from sklearn.metrics.pairwise import cosine_similarity

similarity = cosine_similarity(target_user_row[0].reshape(1, -1), user_pivot_table.loc[:, genres_columns].to_numpy())
similarity

array([[0.83955408, 0.91282376, 0.92956652, 0.90799558, 0.88026299,
        0.60592098, 0.86952207, 0.88234652, 0.83186244, 0.88416004,
        0.86342135, 0.88695919, 0.93563989, 0.81522961, 0.83577593,
        0.91529487, 0.8275372 , 0.88528833, 0.9354192 , 0.64958697,
        0.85960924, 0.85786625, 0.89969035, 0.71519321, 0.8378786 ,
        0.83330694, 0.90274035, 0.63794809, 0.90892425, 0.72906155,
        0.90153311, 0.65414052, 0.96187291, 0.9292602 , 0.91614936,
        0.67939029, 0.95193626, 0.94414619, 0.88204013, 0.66030246,
        0.74750248, 0.93768372, 0.83349687, 0.79786054, 0.8336914 ,
        0.90628679, 0.78083165, 0.92906738, 0.92606823, 0.80911889,
        0.83724697, 0.74106193, 0.89996254, 0.78814376, 0.85909757,
        0.70158836, 0.73810694, 0.87024583, 0.95208885, 0.78006915,
        0.87638795, 0.87402204, 0.67102823, 0.74735191, 0.90635473,
        0.82195601, 0.9456797 , 0.87049233, 0.82374525, 0.81847828,
        0.34742753, 0.94122358, 0.67713604, 0.81

In [20]:
similarity.shape

(1, 1000)

In [21]:
arg_list = np.argsort(similarity[0])[::-1][:10]
arg_list

array([820, 821, 422, 414, 965, 734, 682,  32, 958, 517])

In [22]:
src_userID = arg_list[0]
src_userID = user_pivot_table.iloc[arg_list[0]]["userId"]
src_userID

202948.0

In [23]:
src_userID

202948.0

In [24]:
left = pd.merge(left=ratings.loc[ratings["userId"] == target_userID], right=movies, on="movieId").loc[:, ["userId", "movieId", "title", "rating"]]
left

,userId,movieId,title,rating
0,133290,110,Braveheart (1995),2.0
1,133290,296,Pulp Fiction (1994),1.0
2,133290,356,Forrest Gump (1994),5.0
3,133290,457,"Fugitive, The (1993)",5.0
4,133290,527,Schindler's List (1993),5.0
5,133290,589,Terminator 2: Judgment Day (1991),1.5
6,133290,593,"Silence of the Lambs, The (1991)",1.0
7,133290,899,Singin' in the Rain (1952),4.5
8,133290,903,Vertigo (1958),4.0
9,133290,914,My Fair Lady (1964),4.0


In [25]:
right = pd.merge(left=ratings.loc[ratings["userId"] == src_userID], right=movies, on="movieId").loc[:, ["userId", "movieId", "title", "rating"]]
right

,userId,movieId,title,rating
0,202948,50,"Usual Suspects, The (1995)",4.0
1,202948,170,Hackers (1995),4.0
2,202948,296,Pulp Fiction (1994),3.5
3,202948,318,"Shawshank Redemption, The (1994)",4.0
4,202948,628,Primal Fear (1996),3.0
5,202948,750,Dr. Strangelove or: How I Learned to Stop Worr...,4.0
6,202948,912,Casablanca (1942),3.5
7,202948,1136,Monty Python and the Holy Grail (1975),4.0
8,202948,1186,"Sex, Lies, and Videotape (1989)",3.5
9,202948,1196,Star Wars: Episode V - The Empire Strikes Back...,3.5


In [26]:
pd.merge(left=left, right=right, on="movieId")

,userId_x,movieId,title_x,rating_x,userId_y,title_y,rating_y
0,133290,296,Pulp Fiction (1994),1.0,202948,Pulp Fiction (1994),3.5
1,133290,1198,Raiders of the Lost Ark (Indiana Jones and the...,3.0,202948,Raiders of the Lost Ark (Indiana Jones and the...,3.5
2,133290,58559,"Dark Knight, The (2008)",4.5,202948,"Dark Knight, The (2008)",3.5


In [27]:
left["movieId"]

0        110
1        296
2        356
3        457
4        527
5        589
6        593
7        899
8        903
9        914
10       923
11       953
12      1036
13      1080
14      1193
15      1198
16      1200
17      1233
18      1234
19      1240
20      1244
21      1246
22      1250
23      1262
24      1270
25      1284
26      1291
27      1299
28      1307
29      1393
30      2028
31      2067
32      2183
33      2268
34      2571
35      3545
36      3801
37      4993
38      5952
39      7153
40     40629
41     49530
42     54286
43     58559
44     68358
45     91658
46    109487
Name: movieId, dtype: int64

In [28]:
right["movieId"]

0        50
1       170
2       296
3       318
4       628
5       750
6       912
7      1136
8      1186
9      1196
10     1198
11     1208
12     1217
13     1222
14     1321
15     1409
16     1476
17     1831
18     1957
19     2019
20     2020
21     2420
22     2826
23     2858
24     2905
25     2959
26     3030
27     3148
28     3174
29     3354
30     3386
31     4226
32     4361
33     4873
34     4878
35     4973
36     5291
37     7502
38     7925
39     7981
40    30749
41    58559
42    68237
43    79132
44    80906
45    84152
46    84954
47    85414
Name: movieId, dtype: int64

In [29]:
recommended_movie_ids = list(set(right["movieId"]) - set(left["movieId"]))

In [30]:
right.loc[right["movieId"].isin(recommended_movie_ids)]

,userId,movieId,title,rating
0,202948,50,"Usual Suspects, The (1995)",4.0
1,202948,170,Hackers (1995),4.0
3,202948,318,"Shawshank Redemption, The (1994)",4.0
4,202948,628,Primal Fear (1996),3.0
5,202948,750,Dr. Strangelove or: How I Learned to Stop Worr...,4.0
6,202948,912,Casablanca (1942),3.5
7,202948,1136,Monty Python and the Holy Grail (1975),4.0
8,202948,1186,"Sex, Lies, and Videotape (1989)",3.5
9,202948,1196,Star Wars: Episode V - The Empire Strikes Back...,3.5
11,202948,1208,Apocalypse Now (1979),4.0


In [31]:
target_user_row[0]

array([0.48858447, 0.3652968 , 0.        , 0.        , 0.33333333,
       0.31050228, 0.        , 1.        , 0.06849315, 0.0456621 ,
       0.03652968, 0.11415525, 0.12328767, 0.21461187, 0.38812785,
       0.21917808, 0.4109589 , 0.39269406, 0.        , 0.        ])

In [32]:
genres_columns.to_numpy()

array(['Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime',
       'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'IMAX',
       'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War',
       'Western', 'etc'], dtype=object)

In [44]:
preferred_genre = pd.DataFrame({"preferences":target_user_row[0], "genres":genres_columns})
preferred_genre

,preferences,genres
0,0.488584,Action
1,0.365297,Adventure
2,0.000000,Animation
3,0.000000,Children
4,0.333333,Comedy
5,0.310502,Crime
6,0.000000,Documentary
7,1.000000,Drama
8,0.068493,Fantasy
9,0.045662,Film-Noir


In [45]:
preferred_genre.sort_values(by="preferences", ascending=False)

,preferences,genres
7,1.000000,Drama
0,0.488584,Action
16,0.410959,Thriller
17,0.392694,War
14,0.388128,Romance
1,0.365297,Adventure
4,0.333333,Comedy
5,0.310502,Crime
15,0.219178,Sci-Fi
13,0.214612,Mystery


In [46]:
import plotly.express as px
fig = px.bar(preferred_genre.sort_values(by="preferences", ascending=True), x="preferences", y="genres")
fig.show()